In [1]:
%env AWS_DEFAULT_PROFILE=protecso

env: AWS_DEFAULT_PROFILE=protecso


In [2]:
import boto3
import dynamodbgeo
import uuid
dynamodb = boto3.client('dynamodb', region_name='us-east-1')

In [3]:
config = dynamodbgeo.GeoDataManagerConfiguration(dynamodb, 'climate')
geoDataManager = dynamodbgeo.GeoDataManager(config)

In [4]:
# Pick a hashKeyLength appropriate to your usage
config.hashKeyLength = 5

# Use GeoTableUtil to help construct a CreateTableInput.
table_util = dynamodbgeo.GeoTableUtil(config)
create_table_input=table_util.getCreateTableRequest()

#tweaking the base table parameters as a dict
create_table_input["ProvisionedThroughput"]['ReadCapacityUnits']=5

# Use GeoTableUtil to create the table
table_util.create_table(create_table_input)

Creating table [climate]
Waiting for table [climate] to be created
Table created


In [ ]:
# this is an example of how to put an item in the table
PutItemInput = {
        'Item': {
            'Country': {'S': "Peru"},
            'Capital': {'S': "Lima"},
            'year': {'S': '1000'}
        },
        'ConditionExpression': "attribute_not_exists(hashKey)" # ... Anything else to pass through to `putItem`, eg ConditionExpression

}
geoDataManager.put_Point(dynamodbgeo.PutPointInput(
        dynamodbgeo.GeoPoint(36.879163, 10.243120), # latitude then latitude longitude
         str( uuid.uuid4()), # Use this to ensure uniqueness of the hash/range pairs.
         PutItemInput # pass the dict here
        ))

{'ResponseMetadata': {'RequestId': 'G3N4753TSSO43L8DT3OA4J064FVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 24 Nov 2024 03:36:17 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'G3N4753TSSO43L8DT3OA4J064FVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [26]:
import pandas as pd
# read dataclima.csv
df = pd.read_csv('dataclima.csv')
df.head()

,DISTRITO,AÑO,MES,DÍA,PRECIPITACIÓN,T MAX,T MINIMA
0,VIQUES,2010,1,1,0.0,21.4,7.8
1,VIQUES,2010,1,2,8.8,19.8,7.8
2,VIQUES,2010,1,3,2.5,20.8,8.8
3,VIQUES,2010,1,4,8.5,18.9,9.1
4,VIQUES,2010,1,5,4.8,17.5,9.4


In [6]:
unique_distritos = df['DISTRITO'].unique()
print("Unique values in 'DISTRITO' column:")
print(unique_distritos)

Unique values in 'DISTRITO' column:
['VIQUES' 'EL TAMBO' 'HUACHAC' 'SANTA ROSA DE OCOPA' 'JAUJA' 'RICRAN']


In [18]:
# ensure all columns other tan distrito are numeric
# Ensure all columns other than 'DISTRITO' are numeric
for column in df.columns:
    if column != 'DISTRITO':
        df[column] = pd.to_numeric(df[column], errors='coerce')

# Display the first few rows to verify the changes
df.head()

,DISTRITO,AÑO,MES,DÍA,PRECIPITACIÓN,T MAX,T MINIMA
0,VIQUES,2010,1,1,0.0,21.4,7.8
1,VIQUES,2010,1,2,8.8,19.8,7.8
2,VIQUES,2010,1,3,2.5,20.8,8.8
3,VIQUES,2010,1,4,8.5,18.9,9.1
4,VIQUES,2010,1,5,4.8,17.5,9.4


In [23]:
cities = ["JAUJA", "VIQUES", "EL TAMBO", "HUACHAC", "SANTA ROSA DE OCOPA", "RICRAN"]
locations = {
	"JAUJA": [-11.774541, -75.484053],
	"VIQUES":[-12.159793, -75.231618],
	"EL TAMBO":[-12.059247, -75.214703],
	"HUACHAC":[-12.059247, -75.214703],
	"SANTA ROSA DE OCOPA": [-11.877194, -75.297825],
	"RICRAN": [-11.541854, -75.521856]
}

conditions = {
	"JAUJA": "Jauja es conocida por su clima templado y suelos fértiles, ideales para el cultivo de papa, maíz y hortalizas.",
	"VIQUES": "Viques tiene un clima variado con temporadas de lluvia, adecuado para la agricultura de subsistencia y cultivos de cereales.",
	"EL TAMBO": "El Tambo es una zona agrícola importante con producción de frutas, verduras y flores debido a su clima favorable.",
	"HUACHAC": "Huachac cuenta con un clima templado y es conocida por la producción de papa y otros tubérculos.",
	"SANTA ROSA DE OCOPA": "Santa Rosa de Ocopa tiene un clima fresco y húmedo, ideal para el cultivo de café y otros productos agrícolas.",
	"RICRAN": "Ricran es una zona con clima variado, adecuada para la agricultura de subsistencia y cultivos de cereales."
}


In [19]:
# Select only numeric columns for grouping
numeric_columns = df.select_dtypes(include='number').columns
grouped = df.groupby('DISTRITO')[numeric_columns].mean()
grouped

,AÑO,MES,DÍA,PRECIPITACIÓN,T MAX,T MINIMA
DISTRITO,,,,,,
EL TAMBO,2011.734491,6.194169,15.719603,2.407878,20.121538,4.239888
HUACHAC,2011.363704,6.159821,15.727409,-2.700075,14.708364,-0.455265
JAUJA,2011.101386,6.267764,15.699307,-0.471490,16.422617,0.343141
RICRAN,2011.734491,6.194169,15.719603,2.452916,13.487984,3.296402
SANTA ROSA DE OCOPA,2011.734491,6.194169,15.719603,2.359584,18.703474,4.470596
VIQUES,2011.734491,6.194169,15.719603,2.434739,20.498449,6.134119


In [24]:
# for every city calculate the averages

# Create a list of dictionaries with the required attributes
city_data = []
for city, data in grouped.iterrows():
    city_info = {
        'city': city,
        'description': conditions[city],  # Placeholder description
        'precipitation': data['PRECIPITACIÓN'],
        'maxt': data['T MAX'],
        'mint': data['T MINIMA'],
        'lon': locations[city][1],
        'lat': locations[city][0],
        'meta': 'attributes: city, description, precipitation (mm), maxt (C), mint (C), lon, lat'
    }
    city_data.append(city_info)

# Print the list of dictionaries
for city in city_data:
    print(city)

{'city': 'EL TAMBO', 'description': 'El Tambo es una zona agrícola importante con producción de frutas, verduras y flores debido a su clima favorable.', 'precipitation': np.float64(2.4078784119106698), 'maxt': np.float64(20.12153846153846), 'mint': np.float64(4.239888337468982), 'lon': -75.214703, 'lat': -12.059247, 'meta': 'attributes: city, description, precipitation (mm), maxt (C), mint (C), lon, lat'}
{'city': 'HUACHAC', 'description': 'Huachac cuenta con un clima templado y es conocida por la producción de papa y otros tubérculos.', 'precipitation': np.float64(-2.700074682598955), 'maxt': np.float64(14.708364451082897), 'mint': np.float64(-0.45526512322628854), 'lon': -75.214703, 'lat': -12.059247, 'meta': 'attributes: city, description, precipitation (mm), maxt (C), mint (C), lon, lat'}
{'city': 'JAUJA', 'description': 'Jauja es conocida por su clima templado y suelos fértiles, ideales para el cultivo de papa, maíz y hortalizas.', 'precipitation': np.float64(-0.47149046793760846)

In [25]:
import json
# Save the city_data as a JSON file called climate.json
climate_data = {"climate": city_data}
with open('climate.json', 'w') as json_file:
    json.dump(climate_data, json_file, indent=4)

In [ ]:
import json
# Save the city_data as a JSON file called climate.json
climate_data = {"climate": city_data}
with open('climate.json', 'w') as json_file:
    json.dump(climate_data, json_file, indent=4)